In [1]:
import re # Regular expressions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm # Shows a smart progress meter - wrap iterable with tqdm(iterable)!
import collections 
from collections import Counter, defaultdict

import nltk
from nltk import bigrams, trigrams, ngrams
# requirement: nltk.download("punkt")
import gensim
from gensim.summarization.textcleaner import split_sentences
from gensim.utils import tokenize
from gensim.utils import simple_preprocess 
import itertools
from itertools import chain

import codecs
from IPython.core.display import HTML
plt.style.reload_library()

# To-Do's

Need Gramma rules?



## Import Data-Set

Import the data which sould be used for training and evaluation.

Split the data-set into 90/10 (or other wanted ratio), shuffle indices and return random distribution of documents.


In [2]:
# Later should be improved: Load all data from a Folder and combine it into one data-frame
df_raw = pd.read_csv('2020_06_23_CE-BVerwG_DE_Datensatz.csv') 

In [3]:
def split_dataset(input_text, perc_use, perc_predict): 
    """ input_text: the Data-Frame which sould be processed
        perc_use: the percentage of data which sould be used for training 
        perc_predit: "                                         " prediction analysis and metrics 
    """
    num_doc = len(df_raw) # number of documents
    indices_lst = list(range(num_doc)) # list of (for now) ordered indices of the documents
    np.random.shuffle(indices_lst) # shuffle the lsit randomly 
    
    # define the data-sets for training and prediciton 
    df_use = df_raw.iloc[indices_lst[:round(perc_use*num_doc)]][["Gericht", "doc_id", "text"]] # Later use full data-set 
    df_predict = df_raw.iloc[indices_lst[round((1-perc_predict)*num_doc):]][["Gericht", "doc_id", "text"]] # Later use full data-set 
    
    # print length of data_frames for validation
    print(len(df_use), len(df_predict))
    
    # reset row index after shuffling, append column with original index
    df_use = df_use.reset_index()
    df_predict = df_predict.reset_index()
    
    return df_use, df_predict

In [4]:
# Run code to create data-sets
df_raw_use, df_raw_predict = split_dataset(df_raw, 0.2, 0.01) # Later use full data-set 
#df_raw_use, df_raw_predict = split_dataset(df_raw, 0.9, 0.1) # <- this is the complete data-set

4845 242


## Counting occurrences (helper function)
Count occurrences like “VwGO” (or Tokens of interest) in all decisions either explicitly (exact comparison) or implicitly (ignore case): 

In [5]:
def search_occurrence(input_text, word_string, explicit = True): # helper function
    count = 0
    search = str(word_string)
    
    if explicit == True: # search for the exact matching of strings
        for decision in tqdm(input_text):
            count += len(re.findall(search, decision))
    
    else:
        for decision in tqdm(input_text): # ignore the lower / upper case variance in string
            count += len(re.findall(search, decision, re.IGNORECASE))
    
    print('Counts of “%s“: ' % search, count)

In [6]:
search_occurrence(df_raw_use['text'], "§.*VwGO", False) # False = ignore case


Counts of “§.*VwGO“:  19942


## Define Tokenizer
Use a tokenizer for preprocessing of the given data-set.

Use "case_important = True" for a exact comparison considering the case sensitivity.

In [7]:
# create Sentences
def tokenizer(text_input, min_length = 2, case_important = False, lan = "german"):
    """ text_input: text which should be processed
        min_length: minimum length of the tokens which should be returned
        case_important: Is upper / lower case from importance (False is much faster)
        lan: language of the processed Text
    """
    
    if case_important == False:
        for instance in range(len(text_input)):
            text_input[instance] = simple_preprocess(text_input[instance], min_len = min_length) # use Gensim 
        
    else: # if upper / lower case is from importance
        for instance in range(len(text_input)):

            sentences = nltk.sent_tokenize(text_input[instance],language=lan) #sentence
            sentences_tok = [nltk.tokenize.word_tokenize(sent) for sent in sentences] # Tokenized
            accumulated = list(itertools.chain.from_iterable(sentences_tok)) # Merge List
            
            # Filter the Tokens for short tokens, the list is hereby reversed handeled 
            [accumulated.pop(i) for i in reversed(range(len(accumulated))) if ((len(accumulated[i]) < min_length) and any(xy.isdigit() for xy in accumulated[i]) != True)]                                                                          
                                                                               
            accumulated_sent = accumulated    
            
            text_input[instance] = accumulated_sent # assign tokens back to the Data-frame 


## Create data-set only considering sentences which include key-tokens and tokenize

Input: one column of list of text
output: two columns, 

            one with the processed sentence 
            
            the second, sentence split at the point of occuring reference. If occuring at the beginning, return lst(none)

In [8]:
# Verallgemeinerung für alle searchterms ----- zu langsam...


# # Just include sentences with a searchterm = "§", then tokenize
# def sentence_reduction(text_input, searchterm, lang = "german", case_i = True, min_len = 2):
#     """ text_input: text which will be processed
#         searchterm: the key-token which must occur within the sentence
#         lang = "german": language of the processed text (default = german)
#         case_i = True: is the task case sensitive (default = True)
#         min_len = 2: the minimum length of the tokens (default = 2)
        
#         return: two df columns: 1. processed sentence and 2. split (at searchterm occurance) processed sentence
#     """
#     k = len(text_input)
#     pattern = re.compile(searchterm)
#     text_out = [ [] for _ in range(k)] # gernerate list to write output to
#     for instance in tqdm(range(k)):
#         sentences = nltk.sent_tokenize(text_input[instance],language=lang) #sentence
#         sentences_filtered = [item for item in sentences for i in range(len(item)) if pattern.search(item) != None] # just sentences with §
        
#         # second df column [before][searchterm][after]
#         for sent in sentences_filtered:
#             index_st = pattern.search(sent).span()  #index of searchterm occurance
#             if pattern.search(sent) != None:
#                 break
#             print(index_st)
#             sentences_split_filt = [[], [searchterm], []]
        
        
#         #sentences_split_filt = ...
    
    
    
#         #tokenizer(sentences_split_filt, min_length = min_len, case_important = case_i  )
#         tokenizer(sentences_filtered, min_length = min_len, case_important = case_i  )
        
#         #split_sent_out[instance] = sentences_split_filt
#         text_out[instance] = sentences_filtered
        
#     return text_out #, split_sent_out

In [9]:
    
# df_preproc_regex = df_raw_use.copy() # preprocessed reduced Dataset
# #print(df_preproc_regex["text"])
# df_preproc_regex["text"] = sentence_reduction(df_preproc_regex["text"], "VwGO", lang = "german", min_len = 1)
# print(df_preproc_regex["text"][0])
    
    


In [10]:
# Just include sentences with a searchterm = "§", then tokenize

# same function as above, but only accept searchterm of length one
def sentence_reduction_one_it(text_input, searchterm, lang = "german", case_i = True, min_len = 2):
    """ text_input: text which will be processed
        searchterm: the key-token (in this case only of length one) which must occur within the sentence
        lang = "german": language of the processed text (default = german)
        case_i = True: is the task case sensitive (default = True)
        min_len = 2: the minimum length of the tokens (default = 2)
        
        return: two df columns: 1. processed sentence and 2. split (at searchterm occurance) processed sentence
    """
    k = len(text_input)
    text_out = [ [] for _ in range(k)] # gernerate list to write output to
    sentence_split_out = [ [] for _ in range(k)]
    
    for instance in tqdm(range(k)):
        sentences = nltk.sent_tokenize(text_input[instance],language=lang) #sentence
        sentences_filtered = [item for item in sentences for i in range(len(item)) if item[i] == searchterm] # just sentences with §
        
        # second df column [before][searchterm][after], mehr als ein § pro Satz möglich
        sentence_split = [ [] for _ in range(len(sentences_filtered))]
        num_count = 0
        for item in sentences_filtered:
            index_search_lst = [index for index, elem in enumerate(item) if elem == searchterm ] # list of indices per sentence
            new_item = []
            while(len(index_search_lst) != 0):
                new_item = [[item[0:index_search_lst[-1]]], [searchterm], [item[index_search_lst[-1]+1:]]]
                del index_search_lst[-1]
            
            sentence_split[num_count] = new_item
            num_count += 1
        
        for item in sentence_split:
            for kk in item:
                if kk[0] != searchterm:
                    tokenizer(kk, min_length = min_len, case_important = case_i  )
                    
                    
        tokenizer(sentences_filtered, min_length = min_len, case_important = case_i  )
        
        sentence_split_out[instance] = sentence_split
        text_out[instance] = sentences_filtered
    print(len(text_out), len(sentence_split_out))
    return text_out, sentence_split_out 

### Co-occurance probability (of single Words)

In [46]:
# https://medium.com/analytics-vidhya/a-comprehensive-guide-to-build-your-own-language-model-in-python-5141b3917d6d

# function to create a n-gram from a list of words
def get_ngrams(text_input, order, Keep_duplicates = True):
    """ text_input: Text which is given as an input
        order: The order of the desired n-gram (bi-gram, tri-gram, ...)
        Keep_duplicates: 
    """
    for instance in tqdm(range(len(text_input))):
        sentence = text_input[instance]
        n_grams = ngrams(sentence, order)
        output = [ ' '.join(grams) for grams in n_grams]
        
        if Keep_duplicates == True:
            text_input[instance] = output
        else:
            text_input[instance] = output[::order] #only keep every n-th entry
            
# function to map probabilities of n-grams following each other (co-occurance)            
def n_gram_model_dictionary(text_input, input_ngram_order = 1, mode = "tri", Keep_duplic=False):
    """
    
    """
    
    # Create a placeholder for model
    model = defaultdict(lambda: defaultdict(lambda: 0))
    
    if (input_ngram_order != 1):
        get_ngrams(text_input, order=input_ngram_order, Keep_duplicates=Keep_duplic)
    
    if mode == "tri":
        # Count frequency of co-occurance, 2 words given, output one
        for sentence in tqdm(text_input):
            for word1, word2, word3 in trigrams(sentence, pad_right=True, pad_left=True):
                model[(word1, word2)][word3] += 1
    
    else:
        # Count frequency of co-occurance  
        for sentence in tqdm(text_input):
            for word1, word2 in bigrams(sentence, pad_right=True, pad_left=True):
                model[word1][word2] += 1
         
    return model
    print("Number of dictionary entries:", len(model))
     
# Combine dictionaries and look for overlapping keys and combine the entries
def accumulate_models(list_of_models): 
    model_complete = dict()
    for i in range(len(list_of_models)):
        model_complete = dict(chain(list_of_models[i].items(), model_complete.items()))
    return model_complete
        
def accumulate_models_prob(model_complete):
    # Probability of a word, given the previous two words or word (item)
    for w1_w2 in tqdm(model_complete):
        total_count = float(sum(model_complete[w1_w2].values()))
        for word3 in model_complete[w1_w2]:
            model_complete[w1_w2][word3] /= total_count
    return model_complete


# calculate and accumulate all models 
def model_multi(num_ng_min, num_ng_max, mode1 = "tri", mode2 = "bi", Keep_duplic=False):
    model_comp = {}
    
    if mode1 == "tri":
        for i in tqdm(range(num_ng_max+1)): 
            df_preproc_ngram = df_preproc[["text"]].copy()
            model = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=i, mode= mode1, Keep_duplic=False)
            del df_preproc_ngram 

            model_comp = accumulate_models([model_comp,model])
            del model
            
    if mode2 == "bi":
        for i in tqdm(range(num_ng_max+1)): 
            df_preproc_ngram = df_preproc[["text"]].copy()
            model = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=i, mode= mode2, Keep_duplic=False)
            del df_preproc_ngram 

            model_comp = accumulate_models([model_comp,model])
            del model

    accumulate_models_prob(model_comp)  
    return model_comp


# function to output suggestions of words which should follow the two suggested input-words    
def n_gram_get_suggestion(input_dictionary, suggestion1, suggestion2 = None, num_sugg = 3 ):
    """ input_dictionary: dictionary in which the result should be look up
        suggestion1: first word 
        suggestion2: second word, if None, then return bi-gram model suggestion
        num_sugg = 3: number of output pairs (default = 3, 0 = all possible pairs)
        
        returns: ordered by value list of suggested words which are in the dictionary
    """
    
    # Problem of overwriting overlapping keys solved! --- accumulate first, then calcualte probability!
    
    # If suggestion2 is present
    if suggestion2 != None:
        sorted_toup = sorted(input_dictionary[suggestion1, suggestion2].items(), key = lambda x: x[1], reverse=True)
    else: 
        sorted_toup = sorted(input_dictionary[suggestion1].items(), key = lambda x: x[1], reverse=True)
        
    if num_sugg == 0:
        result = [[k, v] for k, v in sorted_toup]
    else:
        result = [[k, v] for k, v in sorted_toup][:num_sugg]
   # result = sorted(input_dictionary[suggestion1, suggestion2].items(), key = lambda x: x[1], reverse=True)[:num_sugg]
            
    return result


# Regular expressions approach (just for §)

### Search for every Sentence with a key-token (§) and cut the sentence

In [12]:
df_regex = pd.DataFrame(columns = ["Doc_id", "Court", "Book", "Section", "Paragraph", "Doc_index", "Sent_comp", "Sent_split", "Clausel"])

""" Output should be a pd.Dataframe with colums:
    Document id: Where to find the found reference or string occurrence
    Gericht: Which court was the case assigned to
    Book: Which book of legislation (if not found return None)
    Section: Which section within the book (return 0 if non-existent)
    Paragraph: --
    Document index: Index of Clausel (Should be a tupel (start : end))
    sent_comp: complete processed sentence
    sent_split: sentence splittet at searchterm
    Extracted Clausel: The Clausel of length n which was found to be of interest
"""

df_regex = df_raw_use.copy() # copy the corresponding raw row to it 

df_regex.rename(columns={"index":"Index", "Gericht":"Court", "doc_id":"Doc_id", "text":"Sent_comp"}, inplace = True)
df_regex = df_regex[["Index", "Doc_id", "Court", "Sent_comp" ]]
df_regex[["Sent_split"]] = df_raw_use[["index"]]
# add sent_comp to df_regex, and doc_id

# compute reduction of sentences and add to data-frame
#df_regex["Sent_comp"], df_regex["Sent_split"] = sentence_reduction_one_it(df_regex["Sent_comp"], '§', lang = "german",  case_i = True, min_len = 2)

In [13]:
#df_regex["Sent_comp"][1][1], df_regex["Sent_split"][1][1]
df_regex

,Index,Doc_id,Court,Sent_comp,Sent_split
0,23981,BVerwG_2019-12-19_B_7_VR_5_19_NA.txt,BVerwG,BESCHLUSS\nBVerwG 7 VR 5.19 (7 A 8.19)\n\nIn d...,23981
1,13728,BVerwG_2010-05-12_B_22_WD_8_10_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 2 ...,13728
2,5790,BVerwG_2005-04-19_B_8_B_19_05_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 8 ...,5790
3,7327,BVerwG_2006-03-22_B_4_BN_9_06_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 4 ...,7327
4,14360,BVerwG_2010-10-19_B_4_BN_38_10_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 4 ...,14360
...,...,...,...,...,...
4840,2768,BVerwG_2003-08-12_B_8_B_91_03_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 8 ...,2768
4841,1527,BVerwG_2003-01-09_B_3_B_77_02_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 3 ...,1527
4842,864,BVerwG_2002-08-14_B_7_B_25_02_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 7 ...,864
4843,16219,BVerwG_2011-12-22_B_2_B_69_11_NA.txt,BVerwG,BUNDESVERWALTUNGSGERICHT\nBESCHLUSS\nBVerwG 2 ...,16219


### Identify court, book, ... and add sentence after (end of §...) to clausel

--> Goal: create dictionary 

        {identification (VwGo Abs. x...): 1. most propable clausel of lenght 1
                                                                        
                                          2. most propable clausel of lenght 2
                                                                                                       
                                          .....}
                                          
                                          (or three most propable cluase (given with probability!!!!))

In [14]:
# hierarchy = {
#     'section': '§ %d ',
#     'paragraph': 'Abs. %d ',
#     'clause': 'S. %d',
# }

def str_int(string: str) -> int:
    if string == '':
        return None
    else:
        return int(string)
    
# def reference_to_str(reference_dict: dict, level: int = None, book: str = '') -> str:
#     result = ''
#     book = ' ' + book.strip()
#     n = 0
#     for key, expression in hierarchy.items():
#         value = reference_dict[key]
#         if value is not None and (level is None or n <= level):
#             result += expression %  value
#         n += 1
#     result = result.strip() + book
#     result = result.strip()
#     return result

# def find_references(decision_text: str, book: str) -> list:
#     references = re.findall(r'§ (\d+)\W*[Abs.]*\W*(\d+)*\W*[S.]*\W*(\d+)*\W*' + book, decision_text.replace('\n', ''))
#     references = [{'book': book,'section': str_int(m[0]), 'paragraph': str_int(m[1]), 'clause': str_int(m[2])} for m in references]
#     return references

In [15]:
# Mapping of every sentence which icludes a §
def find_references(decision_text: str, book: str) -> list:
    row_lsit = []
    for instance in tqdm(range(len(decision_text))): # for each document  
        row_lsit_2 = []
        for k in range(len(decision_text[instance])):

            references = re.findall(r'§ (\d+)\W*[Abs.]*\W*(\d+)*\W*[S.]*\W*(\d+)*\W*' + book, decision_text[instance][k])
            references = [{'book': book,'section': str_int(m[0]), 'paragraph': str_int(m[1]), 'clause': str_int(m[2])} for m in references]
            row_lsit_2.append(references)
        row_lsit.append(row_lsit_2)
    return row_lsit

In [16]:
#find_references(df_preproc_regex["text"], "VwGO")

### search for Paragraph and add it with n words before and k words after to a list.
### Then throw either N-Gram on it and / or count the explicit probabilities for n words after Paragraph occurance

In [17]:
# also create df with everythin after and everything before § <- throw n-gram on sentence after §

In [18]:
df_preproc_ngram = df_regex.copy()
model_bigram = n_gram_model_dictionary(df_preproc_ngram['Sent_comp'][3], 1, Keep_duplic=False)


### compare word frequency (of lenght 1....n), given the  mapping to book, section, paragraph....

### Alternativ; Then throw tokenizer and n-gram model on it, compute probability

### Idea: invese word-sentence frequency (tf-idf), weighted with the probability for better suggestions

# N-Gram approach (for complete Text)

Throw N-Gram at the complete Corpus after tokinazation.

Use N-Gram for the sentence after a found start of reference. (Combine Regex with N-Gram to save computing cost, more precice solution?)

Approach: 

Preprocess corpus (Tokenization, ...), then check for N-Grams.

Always retrace to the original sentence. append original sentence instead of reduced form.

### Tokenize and save df as preprocessed_df

In [19]:
# Create Copy of Data-set
df_preproc = df_raw_use.copy() # preprocessed reduced Dataset

# Tokenize Data-set
tokenizer(df_preproc['text'], min_length = 2, case_important = False  )
#df_preproc['text'][0]

<ipython-input-7-5e7a0b3cfc05>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_input[instance] = simple_preprocess(text_input[instance], min_len = min_length) # use Gensim


### Problem right now:  get_ngrams has the problem; word_overlapping when comparing following words (or ngrams) -> Try to solve later

In [41]:
model_complete = model_multi(1, 2, mode1 = "tri", mode2 = "bi")

In [45]:
n_gram_get_suggestion(model_complete, "im zusammenhang", "mit der")


[['behaupteten erhebt', 0.014218009478672985],
 ['einer auf', 0.014218009478672985],
 ['auslegung von', 0.014218009478672985]]

In [26]:
## creat shorter version of this above!!!
df_preproc_ngram = df_preproc[["text"]].copy()
model_1gram_tri = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=1, mode="tri", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_2gram_tri = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=2, mode="tri", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_3gram_tri = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=3, mode="tri", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_4gram_tri = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=4, mode="tri", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_5gram_tri = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=5, mode="tri", Keep_duplic=False)
del df_preproc_ngram 

#----------------------------------------------------------------------------------------------------
df_preproc_ngram = df_preproc[["text"]].copy()
model_1gram_bi = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=1, mode="bi", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_2gram_bi = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=2, mode="bi", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_3gram_bi = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=3, mode="bi", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_4gram_bi = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=4, mode="bi", Keep_duplic=False)
del df_preproc_ngram 

df_preproc_ngram = df_preproc[["text"]].copy()
model_5gram_bi = n_gram_model_dictionary(df_preproc_ngram['text'], input_ngram_order=5, mode="bi", Keep_duplic=False)
del df_preproc_ngram 

Exception ignored in: <function tqdm.__del__ at 0x0000018E1CD29700>
Traceback (most recent call last):
  File "C:\Users\David\anaconda3\lib\site-packages\tqdm\std.py", line 1122, in __del__
    self.close()
KeyboardInterrupt: 


In [ ]:
n_gram_get_suggestion(model_2gram_tri,  "im zusammenhang", "mit der")

### Run for multiple n-grams and combine dictionary!

In [ ]:
model_lst = [model_1gram_tri, model_2gram_tri, model_3gram_tri, model_4gram_tri, model_5gram_tri, model_1gram_bi, model_2gram_bi, model_3gram_bi, model_4gram_bi, model_5gram_bi]

model_comp = accumulate_models(model_lst) # with accumulate, also te probabilities are calcualted

In [ ]:
n_gram_get_suggestion(model_comp, "es sich um", num_sugg=10), n_gram_get_suggestion(model_comp, "es sich um", "eine dienststelle im", num_sugg=10)



### To-do
mixed n-gram: given 5 words-chain, what is the next suggested word. (update n_gram_model_dictionary, !!! don't screw up the probabilities)

Or random, define function with arbitrary input length of words (5 words = 3,2 or 2,3 length) and return probabilities

Look up, if full lenght entry are available, if not reduce to shorter entries.


-------------- alternativ

look at every possible combination, take the most probable entry.

## Exporting

Accumulate entires within the df, add accumulation column.





Export DataFrames (raw and accumulated) to .csv



## Evaluation, Metrics

split every sentence after the occurance of § and write a loop

def continious_predict():
    for the k next predicted words (n = k/2 of sentence length)
    for n-gram of the 4 signs/words before, predict the next two words.
    
    
-----do this for n-grams of length 1 to 3 (just predict one next word to the next three words)
-----do this with consideration of only 1 last word......last 4 words

------ for this write function of n-grams that is randomly combinable......3 words....predict next word.....given two words predict next word.....


(repeat everything for random 1000 sentences within the whole document and check precicion achieved)






In [ ]:
# process the prediction data-set in such a way, that prediction can be undertaken
df_raw_predict # the Data-set used for validation, evaluation and testing metrics

## Vizualization

Plot most occuring sentences: compare N-Gram to Regex approach

plot everything in the evaluation metrics